In [2]:
import os
from utils.utils import shuffle_sequence_in_csv

path = os.path.join("data", "promoter", "epd")
positives = [
    os.path.join(path, "human_tata_min128_plus383.fasta"),
    os.path.join(path, "human_tata_min256_plus255.fasta"),
    os.path.join(path, "human_tata_min384_plus127.fasta"),
    #os.path.join(path, "human_tata_min499_plus100.fasta"),
]
negatives = [
    os.path.join(path, "human_non_tata_min128_plus383.fasta"),
    os.path.join(path, "human_non_tata_min256_plus255.fasta"),
    os.path.join(path, "human_non_tata_min384_plus127.fasta"),
    #os.path.join(path, "human_non_tata_min499_plus100.fasta"),
]
generated_negatives = [
    os.path.join(path, "human_generated_non_tata_min128_plus383.fasta"),
    os.path.join(path, "human_generated_non_tata_min256_plus255.fasta"),
    os.path.join(path, "human_generated_non_tata_min384_plus127.fasta"),
]

In [22]:
"""
1. Generate CSV from FASTAs.
"""

import os
from utils.utils import shuffle_sequence_in_csv

path = os.path.join("data", "promoter", "epd")
positives = [
    os.path.join(path, "human_tata_min128_plus383.fasta"),
    os.path.join(path, "human_tata_min256_plus255.fasta"),
    os.path.join(path, "human_tata_min384_plus127.fasta"),
    #os.path.join(path, "human_tata_min499_plus100.fasta"),
]
negatives = [
    os.path.join(path, "human_non_tata_min128_plus383.fasta"),
    os.path.join(path, "human_non_tata_min256_plus255.fasta"),
    os.path.join(path, "human_non_tata_min384_plus127.fasta"),
    #os.path.join(path, "human_non_tata_min499_plus100.fasta"),
]

from data_preparation import generate_csv_from_fasta
for p in positives:
    bname = os.path.basename(p)
    target_path = os.path.join(path, f"{bname.split('.')[0]}.csv")
    generate_csv_from_fasta(p, target_path, 1)
for n in negatives:
    bname = os.path.basename(n)
    target_path = os.path.join(path, f"{bname.split('.')[0]}.csv")
    generate_csv_from_fasta(p, target_path, 0)



In [5]:
# Still on 1., Generate shuffled positives as generated negatives.
import os
from utils.utils import shuffle_sequence_in_csv
import pandas as pd

path = os.path.join("data", "promoter", "epd")
positive_csvs = [
    os.path.join(path, f) for f in ["human_tata_min128_plus383.csv", "human_tata_min256_plus255.csv", "human_tata_min384_plus127.csv"]
]
generated_negatives = [
    os.path.join(path, "human_generated_non_tata_min128_plus383.csv"),
    os.path.join(path, "human_generated_non_tata_min256_plus255.csv"),
    os.path.join(path, "human_generated_non_tata_min384_plus127.csv"),
]

for p, g in zip(positive_csvs, generated_negatives):
    shuffle_sequence_in_csv(p, g)

# Change label into negative.
for g in generated_negatives:
    df = pd.read_csv(g)
    df['label'] = df['label'].apply(lambda x: 0)
    df.to_csv(g, index=False)


In [9]:
"""Merge positive and negative"""
import os
path = os.path.join("data", "promoter", "epd")
positives = [
    os.path.join(path, "human_tata_min128_plus383.csv"),
    os.path.join(path, "human_tata_min256_plus255.csv"),
    os.path.join(path, "human_tata_min384_plus127.csv"),
    #os.path.join(path, "human_tata_min499_plus100.csv"),
]
negatives = [
    os.path.join(path, "human_non_tata_min128_plus383.csv"),
    os.path.join(path, "human_non_tata_min256_plus255.csv"),
    os.path.join(path, "human_non_tata_min384_plus127.csv"),
    #os.path.join(path, "human_non_tata_min499_plus100.csv"),
]

positive = os.path.join(path, "human_tata.csv")
negative = os.path.join(path, "human_non_tata.csv")

from data_preparation import merge_csv
merge_csv(positives, positive)
merge_csv(negatives, negative)

100%|██████████| 3/3 [00:00<00:00, 21.43it/s]


True

In [6]:
"""Merge generated negatives"""
import os
path = os.path.join("data", "promoter", "epd")
generated_negatives = [
    os.path.join(path, "human_generated_non_tata_min128_plus383.csv"),
    os.path.join(path, "human_generated_non_tata_min256_plus255.csv"),
    os.path.join(path, "human_generated_non_tata_min384_plus127.csv"),
    #os.path.join(path, "human_non_tata_min499_plus100.csv"),
]

generated_negative = os.path.join(path, "human_generated_non_tata.csv")

from data_preparation import merge_csv
merge_csv(generated_negatives, generated_negative)

100%|██████████| 3/3 [00:00<00:00, 30.46it/s]


True

In [10]:
"""
Create balanced positive and negative dataset.
"""

import pandas as pd
import os
path = os.path.join("data", "promoter", "epd")
positive = os.path.join(path, "human_tata.csv")
negative = os.path.join(path, "human_non_tata.csv")
positive_balanced = os.path.join(path, "human_tata.balanced.csv")
negative_balanced = os.path.join(path, "human_non_tata.balanced.csv")

size_pos = pd.read_csv(positive).shape[0]
size_neg = pd.read_csv(negative).shape[0]
print(f"size of positive: {size_pos}")
print(f"size of negative: {size_neg}")

if size_pos != size_neg:
    min_val = min([size_neg, size_pos])
    print(f"Length {min_val}")
    path = os.path.join("data", "promoter", "epd")
    positive = os.path.join(path, "human_tata.csv")
    negative = os.path.join(path, "human_non_tata.csv")
    positive_df = pd.read_csv(positive).sample(n=min_val)
    negative_df = pd.read_csv(negative).sample(n=min_val)
    positive_df.to_csv(positive_balanced, index=False)
    negative_df.to_csv(negative_balanced, index=False)

size of positive: 6943
size of negative: 9195
Length 6943


In [ ]:
"""
Create balanced positive and generated dataset.
"""

import pandas as pd
import os
path = os.path.join("data", "promoter", "epd")
positive = os.path.join(path, "human_tata.csv")
generated_negative = os.path.join(path, "human_generated_non_tata.csv")
positive_balanced = os.path.join(path, "human_tata.balanced.csv")
generated_negative_balanced = os.path.join(path, "human_non_tata.balanced.csv")

size_pos = pd.read_csv(positive).shape[0]
size_neg = pd.read_csv(negative).shape[0]
print(f"size of positive: {size_pos}")
print(f"size of negative: {size_neg}")

if size_pos != size_neg:
    min_val = min([size_neg, size_pos])
    print(f"Length {min_val}")
    path = os.path.join("data", "promoter", "epd")
    positive = os.path.join(path, "human_tata.csv")
    negative = os.path.join(path, "human_non_tata.csv")
    positive_df = pd.read_csv(positive).sample(n=min_val)
    negative_df = pd.read_csv(negative).sample(n=min_val)
    positive_df.to_csv(positive_balanced, index=False)
    negative_df.to_csv(negative_balanced, index=False)

In [11]:
"""Create train and validation for positive and negative"""
from data_preparation import split_csv
path = os.path.join("data", "promoter", "epd")
positive = os.path.join(path, "human_tata.csv")
positive_train = os.path.join(path, "human_tata.train.csv")
positive_validation = os.path.join(path, "human_tata.validation.csv")
negative = os.path.join(path, "human_non_tata.csv")
negative_train = os.path.join(path, "human_non_tata.train.csv")
negative_validation = os.path.join(path, "human_non_tata.validation.csv")
split_csv(positive, [0.8, 0.2], [positive_train, positive_validation])
split_csv(negative, [0.8, 0.2], [negative_train, negative_validation])

"""Merge training and validation"""
from data_preparation import merge_csv
train = os.path.join(path, "promoter.train.csv")
validation = os.path.join(path, "promoter.validation.csv")
merge_csv([positive_train, negative_train], train)
merge_csv([positive_validation, negative_validation], validation)

100%|██████████| 2/2 [00:00<00:00, 37.04it/s]


True

In [8]:
"""Create train and validation for generated negative"""
from data_preparation import split_csv
import os
path = os.path.join("data", "promoter", "epd")
generated_negative = os.path.join(path, "human_generated_non_tata.csv")
generated_negative_train = os.path.join(path, "human_generated_non_tata.train.csv")
generated_negative_validation = os.path.join(path, "human_generated_non_tata.validation.csv")
split_csv(generated_negative, [0.8, 0.2], [generated_negative_train, generated_negative_validation])

In [12]:
"""Create train and validation for balanced dataset."""
from data_preparation import split_csv
path = os.path.join("data", "promoter", "epd")
positive = os.path.join(path, "human_tata.balanced.csv")
positive_train = os.path.join(path, "human_tata.balanced.train.csv")
positive_validation = os.path.join(path, "human_tata.balanced.validation.csv")
negative = os.path.join(path, "human_non_tata.balanced.csv")
negative_train = os.path.join(path, "human_non_tata.balanced.train.csv")
negative_validation = os.path.join(path, "human_non_tata.balanced.validation.csv")
split_csv(positive, [0.8, 0.2], [positive_train, positive_validation])
split_csv(negative, [0.8, 0.2], [negative_train, negative_validation])

"""Merge training and validation"""
from data_preparation import merge_csv
train = os.path.join(path, "promoter_train.balanced.csv")
validation = os.path.join(path, "promoter_validation.balanced.csv")
merge_csv([positive_train, negative_train], train)
merge_csv([positive_validation, negative_validation], validation)

100%|██████████| 2/2 [00:00<00:00, 42.46it/s]


True

In [23]:
"""
This step and the following are necessary if you need to create kmer version of sequence.
Generate kmer format.
"""
import os
path = os.path.join("data", "promoter", "epd")
positives = [
    os.path.join(path, "human_tata_min128_plus383.csv"),
    os.path.join(path, "human_tata_min256_plus255.csv"),
    os.path.join(path, "human_tata_min384_plus127.csv"),
    #os.path.join(path, "human_tata_min499_plus100.csv"),
]
negatives = [
    os.path.join(path, "human_non_tata_min128_plus383.csv"),
    os.path.join(path, "human_non_tata_min256_plus255.csv"),
    os.path.join(path, "human_non_tata_min384_plus127.csv"),
    #os.path.join(path, "human_non_tata_min499_plus100.csv"),
]

from data_preparation import generate_kmer_csv
for p in positives:
    bname = os.path.basename(p)
    target_path = os.path.join(path, f"{bname.split('.')[0]}.kmer.csv")
    generate_kmer_csv(p, target_path)
for n in negatives:
    bname = os.path.basename(n)
    target_path = os.path.join(path, f"{bname.split('.')[0]}.kmer.csv")
    generate_kmer_csv(n, target_path)


In [10]:
"""This step and the following are necessary if you need to create kmer version of generated negative sequence."""
import os
from data_preparation import generate_kmer_csv

path = os.path.join("data", "promoter", "epd")
generated_negatives = [
    os.path.join(path, "human_generated_non_tata_min128_plus383.csv"),
    os.path.join(path, "human_generated_non_tata_min256_plus255.csv"),
    os.path.join(path, "human_generated_non_tata_min384_plus127.csv"),
]
for n in generated_negatives:
    bname = os.path.basename(n)
    target_path = os.path.join(path, f"{bname.split('.')[0]}.kmer.csv")
    generate_kmer_csv(n, target_path)


In [11]:
"""Generate kmer version of generated negative"""
import os
from data_preparation import generate_kmer_csv

path = os.path.join("data", "promoter", "epd")
srcs = [
    os.path.join(path, "human_generated_non_tata.csv"),
    os.path.join(path, "human_generated_non_tata.train.csv"),
    os.path.join(path, "human_generated_non_tata.validation.csv"),
]

dests = [
    os.path.join(path, "human_generated_non_tata.kmer.csv"),
    os.path.join(path, "human_generated_non_tata.kmer.train.csv"),
    os.path.join(path, "human_generated_non_tata.kmer.validation.csv"),
]

for s, d in zip(srcs, dests):
    generate_kmer_csv(s, d)


In [14]:
import os
from data_preparation import generate_kmer_csv

path = os.path.join("data", "promoter", "epd")
srcs = [
    os.path.join(path, "human_tata.balanced.train.csv"),
    os.path.join(path, "human_tata.balanced.validation.csv"),
]
dests = [
    os.path.join(path, "human_tata.balanced.kmer.train.csv"),
    os.path.join(path, "human_tata.balanced.kmer.validation.csv"),
]

for s, d in zip(srcs, dests):
    generate_kmer_csv(s, d)

In [24]:
"""
Split into train and validation.
"""
import os
path = os.path.join("data", "promoter", "epd")
positives = [
    os.path.join(path, "human_tata_min128_plus383.kmer.csv"),
    os.path.join(path, "human_tata_min256_plus255.kmer.csv"),
    os.path.join(path, "human_tata_min384_plus127.kmer.csv"),
    #os.path.join(path, "human_tata_min499_plus100.kmer.csv"),
]
negatives = [
    os.path.join(path, "human_non_tata_min128_plus383.kmer.csv"),
    os.path.join(path, "human_non_tata_min256_plus255.kmer.csv"),
    os.path.join(path, "human_non_tata_min384_plus127.kmer.csv"),
    #os.path.join(path, "human_non_tata_min499_plus100.kmer.csv"),
]

import pandas as pd
positives_negatives = [positives, negatives]
for pn in positives_negatives:
    for p in pn:
        df = pd.read_csv(p)
        train_df = df.sample(frac=0.8)
        validation_df = df.drop(train_df.index)
        train_df.to_csv(
            os.path.join(path, f"{'.'.join(os.path.basename(p).split('.')[0:2])}.train.csv"),
            index=False
        )
        validation_df.to_csv(
            os.path.join(path, f"{'.'.join(os.path.basename(p).split('.')[0:2])}.validation.csv"),
            index=False
        )

In [25]:
"""
Merge CSV which is already in kmer version.
"""
import os
from data_preparation import merge_csv
path = os.path.join("data", "promoter", "epd")
trains = [
    os.path.join(path, "human_tata_min128_plus383.kmer.train.csv"),
    os.path.join(path, "human_tata_min256_plus255.kmer.train.csv"),
    os.path.join(path, "human_tata_min384_plus127.kmer.train.csv"),
    #os.path.join(path, "human_tata_min499_plus100.kmer.train.csv"),
    os.path.join(path, "human_non_tata_min128_plus383.kmer.train.csv"),
    os.path.join(path, "human_non_tata_min256_plus255.kmer.train.csv"),
    os.path.join(path, "human_non_tata_min384_plus127.kmer.train.csv"),
    #os.path.join(path, "human_non_tata_min499_plus100.kmer.train.csv"),
]
merge_csv(trains, os.path.join(path, "human_tata.kmer.train.csv"))
validations = [
    os.path.join(path, "human_tata_min128_plus383.kmer.validation.csv"),
    os.path.join(path, "human_tata_min256_plus255.kmer.validation.csv"),
    os.path.join(path, "human_tata_min384_plus127.kmer.validation.csv"),
    #os.path.join(path, "human_tata_min499_plus100.kmer.validation.csv"),
    os.path.join(path, "human_non_tata_min128_plus383.kmer.validation.csv"),
    os.path.join(path, "human_non_tata_min256_plus255.kmer.validation.csv"),
    os.path.join(path, "human_non_tata_min384_plus127.kmer.validation.csv"),
    #os.path.join(path, "human_non_tata_min499_plus100.kmer.validation.csv"),
]
merge_csv(trains, os.path.join(path, "human_tata.kmer.validation.csv"))


100%|██████████| 6/6 [00:00<00:00, 11.37it/s]


True

In [16]:
s = "ATT TTG TGG GGG GGG GGC GCA CAA AAT ATG TGC GCT CTC TCC CCT CTG TGT GTT TTA TAT ATC TCC CCA CAC ACA CAC ACA CAG AGT GTG TGT GTC TCA CAT ATT TTT TTA TAA AAG AGG GGG GGT GTG TGA GAC ACA CAT ATT TTA TAG AGA GAG AGA GAT ATG TGG GGC GCA CAT ATG TGA GAA AAG AGT GTG TGC GCC CCC CCA CAG AGC GCA CAC ACA CAG AGA GAG AGC GCA CAG AGG GGT GTG TGC GCT CTC TCA CAC ACA CAG AGG GGC GCT CTG TGT GTT TTG TGG GGG GGT GTC TCC CCC CCC CCA CAC ACC CCC CCT CTA TAC ACT CTG TGA GAC ACA CAC ACA CAC ACC CCA CAA AAG AGC GCA CAC ACA CAC ACG CGT GTC TCT CTG TGC GCC CCC CCT CTG TGC GCC CCA CAT ATG TGG GGT GTG TGG GGG GGG GGA GAG AGC GCA CAA AAA AAC ACA CAA AAT ATG TGT GTT TTA TAT ATG TGG GGT GTT TTC TCC CCG CGA GAC ACT CTC TCC CCC CCC CCG CGG GGG GGA GAG AGG GGC GCC CCA CAG AGG GGC GCC CCG CGG GGG GGC GCA CAT ATT TTT TTC TCA CAC ACT CTG TGT GTA TAG AGG GGA GAT ATG TGT GTT TTG TGA GAA AAG AGC GCC CCG CGC GCA CAG AGG GGC GCA CAG AGT GTC TCG CGT GTC TCG CGC GCT CTG TGC GCG CGA GAG AGA GAA AAG AGA GAG AGA GAG AGC GCC CCT CTG TGG GGC GCT CTC TCT CTG TGA GAT ATG TGT GTC TCC CCA CAC ACT CTG TGT GTC TCT CTT TTC TCC CCA CAG AGG GGG GGA GAG AGC GCC CCT CTG TGG GGT GTG TGG GGG GGA GAA AAG AGG GGA GAA AAG AGG GGA GAG AGT GTG TGG GGG GGC GCA CAG AGC GCG CGG GGC GCC CCC CCC CCT CTC TCG CGC GCT CTC TCT CTG TGC GCG CGG GGG GGC GCC CCT CTC TCT CTC TCC CCT CTG TGC GCC CCC CCT CTT TTT TTG TGT GTA TAC ACT CTC TCC CCA CAC ACG CGA GAG AGG GGT GTG TGT GTG TGA GAG AGG GGA GAA AAG AGT GTT TTG TGC GCC CCG CGG GGG GGT GTC TCA CAC ACC CCC CCA CAG AGC GCA CAG AGA GAG AGG GGG GGA GAG AGA GAG AGG GGT GTG TGA GAT ATG TGT GTC TCC CCC CCC CCT CTC TCT CTG TGT GTT TTC TCT CTT TTC TCC CCT CTA TAA AAT ATT TTT TTG TGT GTA TAG AGC GCT CTA TAA AAT ATC TCA CAG AGA GAT ATT TTC TCT CTG TGC GCC CCT CTG TGC GCA CAG AGC GCC CCA CAG AGG GGC GCT CTC TCT CTC TCT CTG TGG GGG GGG GGA GAT ATG TGA GAA AAT ATC TCT CTA TAA AAT ATA TAA AAA AAG AGG GGA GAG AGG GGC GCG CGG GGC GCG CGA GAC ACC CCT CTG TGG GGA GAG AGA GAG AGG GGA GAG AGG GGA GAC ACA CAG AGA GAC ACA CAA AAC ACA CAA AAG AGT GTG TGA GAC ACC CCA CAG AGG GGG GGG GGG GGG GGA GAC ACA CAG AGC GCA CAC ACT CTT TTC TCT CTT TTT TTC TCA CAG AGT GTA TAC ACA CAT ATT TTT TTT TTC TCC CCT CTG TGT GTT TTA TAG AGA GAC ACT CTC TCG CGG GGG GGG GGT GTA TAT ATT TTA TAG AGA GAG AGA GAC ACC CCC CCA CAA AAA AAG AGA GAT ATA TAC ACC CCC CCA CAA AAG AGG GGA GAG AGG GGA GAG AGT GTG TGA GAG AGA GAG AGC GCG"
s = s.split(' ')
len(s)

598

In [16]:
"""Generate promoter data from positive and generated negative data in kmer version."""
import os
from data_preparation import merge_csv

path = os.path.join("data", "promoter", "epd")
train_srcs = [
    os.path.join(path, "human_tata.balanced.kmer.train.csv"),
    os.path.join(path, "human_generated_non_tata.kmer.train.csv")
]
train_dest = os.path.join(path, "promoter_train.balanced.generated_negative.kmer.csv")
merge_csv(train_srcs, train_dest)
validation_srcs = [
    os.path.join(path, "human_tata.balanced.kmer.validation.csv"),
    os.path.join(path, "human_generated_non_tata.kmer.validation.csv")
]
validation_dest = os.path.join(path, "promoter_validation.balanced.generated_negative.kmer.csv")
merge_csv(validation_srcs, validation_dest)


100%|██████████| 2/2 [00:00<00:00, 21.50it/s]


True

In [17]:
"""
Generate promoter data from positive and generate negative data in normal format.
"""
import os
from data_preparation import merge_csv

path = os.path.join("data", "promoter", "epd")
train_srcs = [
    os.path.join(path, "human_tata.balanced.train.csv"),
    os.path.join(path, "human_generated_non_tata.train.csv")
]
train_dest = os.path.join(path, "promoter_train.balanced.generated_negative.csv")
merge_csv(train_srcs, train_dest)
validation_srcs = [
    os.path.join(path, "human_tata.balanced.validation.csv"),
    os.path.join(path, "human_generated_non_tata.validation.csv")
]
validation_dest = os.path.join(path, "promoter_validation.balanced.generated_negative.csv")
merge_csv(validation_srcs, validation_dest)

100%|██████████| 2/2 [00:00<00:00, 81.56it/s]


True